# Running Inference with the Mistral 7B Model

In this notebook, we'll set up and utilize the Mistral 7B "Instruct" model. Our primary objective is to perform inference on this model and experiment with various completions.


### Setup Runtime
For fine-tuning Llama, a GPU instance is essential. Follow the directions below:

1. Go to `Runtime` (located in the top menu bar).
2. Select `Change Runtime Type`.
3. Choose `T4 GPU` (or a comparable option).


### Install Transformers Library from GitHub

The code below installs the `transformers` library directly from the HuggingFace GitHub repository.



In [ ]:
!pip install git+https://github.com/huggingface/transformers

### Installing Additional Libraries

The following commands install several libraries:

- `accelerate`: A library from HuggingFace that aids in utilizing hardware accelerators like GPUs and TPUs more efficiently.
- `bitsandbytes`: Provides fast gradient compression, beneficial for accelerated training, particularly in distributed scenarios.
- `sentencepiece`: A library for Neural Network-based text processing, often used in tokenization processes for language models.

The `-q` flag ensures a quiet installation, minimizing the log output.



In [ ]:
!pip install -q peft  accelerate bitsandbytes safetensors

In [ ]:
!pip install sentencepiece


### Model Initialization and Setup

In this section:

- **torch**: The PyTorch library is imported, which will be used for tensor operations and to leverage GPU acceleration.
  
- **AutoModelForCausalLM**: From the HuggingFace Transformers library, this class provides an interface to load models designed for causal language modeling. Causal language models predict the next token in a sequence.

- **AutoTokenizer**: This class is used to load tokenizers that can convert text into tokens suitable for the input of a transformer model.

- `model_name`: Defines the identifier for the model we want to load. In this case, we're using the sharded version of the Mistral-7B model named [`"filipealmeida/Mistral-7B-Instruct-v0.1-sharded"`](https://huggingface.co/filipealmeida/Mistral-7B-Instruct-v0.1-sharded).


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers

model_name = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"


### Setting up the BitsAndBytes Configuration

The code block below configures the `BitsAndBytes` quantization settings, which are designed to optimize model performance by reducing the memory requirements of the model parameters:

- `load_in_4bit`: This flag, set to `True`, instructs the model to load its weights in 4-bit quantization. This can reduce memory usage significantly, allowing for larger models to fit into memory.

- `bnb_4bit_use_double_quant`: When set to `True`, this flag enables double quantization, which can further enhance the efficiency of 4-bit quantization.

- `bnb_4bit_quant_type`: Specifies the type of 4-bit quantization to use. The value `"nf4"` represents a specific form of quantization, but details on this are needed for a more complete description.

- `bnb_4bit_compute_dtype`: This defines the data type to use for computations when the model weights are quantized. Here, `torch.bfloat16` is used, which is a 16-bit floating point representation that offers a balance between precision and memory usage.


In [ ]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

### Loading the Pretrained Model with Quantization

The code below is responsible for loading our pretrained Mistral-7B model, utilizing the previously configured `BitsAndBytes` quantization settings:

- `model_name`: Specifies the identifier for the pretrained model we want to load, which we've previously set to the sharded version of the Mistral-7B model.

- `load_in_4bit`: With this set to `True`, the model loads its weights using 4-bit quantization, which significantly reduces memory requirements.

- `torch_dtype`: Specifies the data type for tensor computations. We've set it to `torch.bfloat16` to strike a balance between memory efficiency and computational precision.

- `quantization_config`: We provide the `BitsAndBytes` configuration (`bnb_config`) established in the previous step to apply the specified quantization settings during model loading.

By leveraging these settings, the model is loaded in a memory-optimized manner, ensuring that even large models like Mistral-7B can be effectively used in constrained environments.


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)


### Tokenizer Initialization and Configuration

1. **Initialize the Tokenizer**: Using the `AutoTokenizer` class from the `transformers` library, we initialize a tokenizer corresponding to our predefined model, `model_name`.
2. **Set Beginning of Sequence Token**: The `bos_token_id` is set to `1`, designating this token ID as the beginning of a sequence.
3. **Define Stop Tokens**: We define a list of token IDs, `stop_token_ids`, that signify stopping points in token sequences. Here, the token ID `0` is considered a stop token.
4. **Confirmation Print**: A print statement confirms the successful loading of the model into memory.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.bos_token_id = 1
stop_token_ids = [0]

print(f"Successfully loaded the model {model_name} into memory")


### Generating Text with the Model 🚀

1. **Define Instruction Text** 📝: We set up our instruction text in the `text` variable. Remember to replace `~Add your instrunctions here~` with the actual content you wish to provide.
2. **Tokenize Input Text** 🔢: Using our previously initialized `tokenizer`, we convert the instruction text into its tokenized form with `return_tensors="pt"` to get the output as PyTorch tensors.
3. **Model Inference** 🤖: With our tokenized input, we run the model's `generate` function to produce an output. We specify a maximum of 200 new tokens to be generated and enable sampling for diverse outputs.
4. **Decode the Output** 📄: The generated token IDs are decoded back into human-readable text using `tokenizer.batch_decode`.
5. **Print the Result** 🖨️: We display the model's generated output for review.



In [ ]:

text = "[INST] ~Add your instrunctions here~ [/INST]"

encoded = tokenizer(text, return_tensors="pt", add_special_tokens=False)
model_input = encoded
generated_ids = model.generate(**model_input, max_new_tokens=200, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])